# Group Assignment TM10007 - Machine Learning
Group 3
Noor Borren
Puck Groen 4470044
Lucy Knöps 
Judith Sluijter

In [ ]:
# Import
from hn.load_data import load_data

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn import model_selection, metrics, feature_selection, preprocessing, neighbors, decomposition, svm
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import roc_auc_score

In [ ]:
# Functions

def data_preprocessing_pca(X_train, X_validation):
    ''' Data preprocessing: first scaling and then PCA with a optimized number of components '''

    # 1. Scaling 
    scaler = preprocessing.StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_validation_scaled = scaler.transform(X_validation)

    # 2. Principle component analysis
    # Using the cumulative summation of the explained variance, we concluded that in
    # order to retain 95% of the variance 30 components are needed.

    pca = PCA(n_components=30)
    pca.fit(X_train_scaled)
    X_train_pca = pca.transform(X_train_scaled)
    X_validation_pca = pca.transform(X_validation_scaled)

    return X_train_pca, X_validation_pca


def data_preprocessing_uni(X_train, y_train, X_validation):
    '''Data preprocessing: first scaling and then univariate analysis'''

    # 1. Scaling 
    scaler = preprocessing.RobustScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    df_X_train_scaled = pd.DataFrame(X_train_scaled)
    X_validation_scaled = scaler.transform(X_validation)
    df_X_validation_scaled = pd.DataFrame(X_validation_scaled)

    bestfeatures = SelectKBest(score_func=mutual_info_classif, k=10)
    fit = bestfeatures.fit(df_X_train_scaled, y_train)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(df_X_train_scaled.columns)
    # Concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  # Naming the dataframe columns
    best_features = featureScores.nlargest(10,'Score')['Specs']

    X_train_uni = df_X_train_scaled[best_features]
    X_validation_uni = df_X_validation_scaled[best_features]

    return X_train_uni, X_validation_uni


def plot_learning_curve(estimator, title, X, y, axes, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """

    axes.set_title(title)
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Training examples")
    axes.set_ylabel("Score")

    train_sizes, train_scores, test_scores  = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes.legend(loc="best")

    return plt

# Data Loading

In [ ]:
# Loading of the data 
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of features: {len(data.columns)-1}')
y_labels = data['label']
del data['label']

y = preprocessing.label_binarize(y_labels, ['T12', 'T34']) 
# 0 stands for T12 and 1 for T34
y = [i[0] for i in y]
y = np.array(y)

# Split data in a train and test set
split_X_train, split_X_test, split_y_train, split_y_test = train_test_split(data, y, stratify=y, test_size=0.2)

# Preprocessing of the data
For the preprocessing of the data, we always started with scaling the data. After the scaling we tried different preprocessing steps. First, we tried the Feature ranking with recursive feature elimination, this is shown in appendix 1.a. This didn't work as well as we wanted to, so we tried two different preprocessing steps: the Principal Component Analysis (PCA) and the Univariate analysis. These two options for datapreprocessing are shown in the cell with functions (data_preprocessing_pca, data_preprocessing_uni). We applied both of the preprocessing functions to different classifiers and compared the results. 

# Classifiers
After the preprocessing we applied different classifiers to see which one gave the best result. We applied the k-Nearest Neighbor classifier (kNN), the Logistic Regression classifier, the Random Forest classifier and the Support Vector Machine classifier. 

# kNN Classifier
We tried the kNN classifier with the two different preprocessing methods. First with PCA and second with Univariate Analysis, the code is provided below.

In [ ]:
# Classifier: kNN
# Preprocessing: PCA
# Dataset: training & validation

cv_4fold = StratifiedKFold(n_splits=4, shuffle=True)

k_list = list(range(1, 50, 2))
all_train = []
all_validation = []
for _ in range(0,20):
    for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
        train_scores = []
        validation_scores = []
        X_validation = split_X_train.iloc[validation_index]
        y_validation = split_y_train[validation_index]
        X_train = split_X_train.iloc[training_index]
        y_train = split_y_train[training_index]

        # Preprocessing using PCA
        X_train_pca, X_validation_pca = data_preprocessing_pca(X_train, X_validation)

        for k in k_list:
            clf_knn = KNeighborsClassifier(n_neighbors=k)
            clf_knn.fit(X_train_pca, y_train)

            # Test the classifier on the training data and plot
            train_proba = clf_knn.predict_proba(X_train_pca)[:, 1]
            validation_proba = clf_knn.predict_proba(X_validation_pca)[:, 1]
            
            score_train = roc_auc_score(y_train, train_proba)
            score_validation = roc_auc_score(y_validation, validation_proba)

            train_scores.append(score_train)
            validation_scores.append(score_validation)

        all_train.append(train_scores)
        all_validation.append(validation_scores)

# Create numpy array of scores and calculate the mean and std
all_train = np.array(all_train)
all_validation = np.array(all_validation)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

validation_scores_mean = all_validation.mean(axis=0)
validation_scores_std = all_validation.std(axis=0)

# Plot the learning curve (mean scores and std as shading)
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.grid()
ax.fill_between(k_list, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
ax.fill_between(k_list, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha=0.1,
                     color="g")
ax.plot(k_list, train_scores_mean, 'o-', color="r",
        label="Training score")
ax.plot(k_list, validation_scores_mean, 'o-', color="g",
        label="Validation score")

In [ ]:
# Classifier: kNN
# Preprocessing: PCA
# Dataset: training & validation

# kNN will be plotted for the scoring of the classifiers

k_list = list(range(1, 50, 2))
all_train = []
all_validation = []
for _ in range(0,20):
    for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
        train_scores = []
        validation_scores = []
        X_validation = split_X_train.iloc[validation_index]
        y_validation = split_y_train[validation_index]
        X_train = split_X_train.iloc[training_index]
        y_train = split_y_train[training_index]

        # Preprocessing using PCA
        X_train_pca, X_validation_pca = data_preprocessing_pca(X_train, X_validation)

        for k in k_list:
            clf_knn = KNeighborsClassifier(n_neighbors=k)
            clf_knn.fit(X_train_pca, y_train)

            # Test the classifier on the training data and plot
            score_train = clf_knn.score(X_train_pca, y_train)
            score_validation = clf_knn.score(X_validation_pca, y_validation)            

            train_scores.append(score_train)
            validation_scores.append(score_validation)

        all_train.append(train_scores)
        all_validation.append(validation_scores)

# Create numpy array of scores and calculate the mean and std
all_train = np.array(all_train)
all_validation = np.array(all_validation)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

validation_scores_mean = all_validation.mean(axis=0)
validation_scores_std = all_validation.std(axis=0)

# Plot the learning curves (mean scores and the std as shading)
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.grid()
ax.fill_between(k_list, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
ax.fill_between(k_list, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha=0.1,
                     color="g")
ax.plot(k_list, train_scores_mean, 'o-', color="r",
        label="Training score")
ax.plot(k_list, validation_scores_mean, 'o-', color="g",
        label="Validation score")

The above graphics show the trend of the kNN. Another method for obtaining the exact number of neighbors which works best in a certain fold is using a GridSearch. The use of a GridSearch is shown below. 

In [ ]:
# Classifier: kNN
# Preprocessing: PCA
# Dataset: training & validation
# Use of GridSearchCV to determine k 

clf_knn = KNeighborsClassifier()

# Specify the search range, this could be multiple parameters for more complex classifiers
parameters = {"n_neighbors": list(range(1, 50, 2))}

cv_5fold = StratifiedKFold(n_splits=4, shuffle=True)
grid_search = GridSearchCV(clf_knn, parameters, cv=cv_5fold, scoring='roc_auc')

result = []
for _ in range(20):
    X_train_pca, X_validation_pca = data_preprocessing_pca(split_X_train, X_validation)
    grid_search.fit(X_train_pca, split_y_train)
    print(grid_search.best_params_)

As you can see in the results above, the optimal number of neighbors varies a lot between folds. Below we applied the Gridsearch to the method and applied it to the test set.

In [ ]:
# Classifier: kNN
# Preprocessing: PCA
# Dataset: test
# Use of GridSearchCV to determine k

clf_knn = KNeighborsClassifier()

# Specify the search range, this could be multiple parameters for more complex classifiers
parameters = {"n_neighbors": list(range(1, 50, 2))}

cv_4fold = StratifiedKFold(n_splits=4, shuffle=True)
grid_search = GridSearchCV(clf_knn, parameters, cv=cv_4fold, scoring='roc_auc')

train_all = []
test_all = []

for _ in range(10):
    for train_index, test_index in cv_4fold.split(data, y):
        train_scores = []
        test_scores = []
        X_test = data.iloc[test_index]
        y_test = y[test_index]
        X_train = data.iloc[train_index]
        y_train = y[train_index]

        # Data preprocessing
        X_train_pca, X_test_pca = data_preprocessing_pca(X_train, X_test)
        grid_search.fit(X_train_pca, y_train) 
        clf_knn = grid_search.best_estimator_

        # Test
        train_proba = clf_knn.predict_proba(X_train_pca)[:, 1]
        test_proba = clf_knn.predict_proba(X_test_pca)[:, 1]

        score_train = roc_auc_score(y_train, train_proba)
        score_test = roc_auc_score(y_test, test_proba)

        train_scores.append(score_train)
        test_scores.append(score_test)

    train_all.append(train_scores)
    test_all.append(test_scores)

# Create numpy array of scores and calculate the mean and std
all_train = np.array(train_all)
all_test = np.array(test_all)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

test_scores_mean = all_test.mean(axis=0)
test_scores_std = all_test.std(axis=0)
print(f'AUC: {test_scores_mean} +- {test_scores_std}')

The above results are quite good if we want to achieve an AUC of 70% or higher. However, the AUC varies and there is also a big variance in the standard deviation. Therefore, we want to adapt our model to achieve better and more robust results. We will now look at the Univariate Analysis as a preprocessing step. Below, only the train-validation curve is plotted.

In [ ]:
# Classifier: kNN
# Preprocessing: Univariate Analysis
# Dataset: training & validation

# AUROC of different values for K in kNN

cv_4fold = StratifiedKFold(n_splits=4, shuffle=True)

k_list = list(range(1, 50, 2))
all_train = []
all_test = []
for _ in range(0,20):
    for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
        train_scores = []
        test_scores = []
        X_validation = split_X_train.iloc[validation_index]
        y_validation = split_y_train[validation_index]
        X_train = split_X_train.iloc[training_index]
        y_train = split_y_train[training_index]

        # Preprocessing using PCA
        X_train_uni, X_validation_uni = data_preprocessing_uni(X_train, y_train, X_validation)

        for k in k_list:
            clf_knn = neighbors.KNeighborsClassifier(n_neighbors=k)
            clf_knn.fit(X_train_uni, y_train)

            # Test the classifier on the training data and plot
            train_proba = clf_knn.predict_proba(X_train_uni)[:, 1]
            test_proba = clf_knn.predict_proba(X_validation_uni)[:, 1]
            
            score_train = roc_auc_score(y_train, train_proba)
            score_test = roc_auc_score(y_validation, test_proba)

            train_scores.append(score_train)
            test_scores.append(score_test)

        all_train.append(train_scores)
        all_test.append(test_scores)

# Create numpy array of scores and calculate the mean and std
all_train = np.array(all_train)
all_test = np.array(all_test)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

test_scores_mean = all_test.mean(axis=0)
test_scores_std = all_test.std(axis=0)

# Plot the learning curves (mean scores and the std as shading)
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.grid()
ax.fill_between(k_list, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
ax.fill_between(k_list, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1,
                     color="g")
ax.plot(k_list, train_scores_mean, 'o-', color="r",
        label="Training score")
ax.plot(k_list, test_scores_mean, 'o-', color="g",
        label="Test score")

The graph above shows a higher AUROC with an AUC score above the 0.7. Next, the hyperparameter tuning using GridSearch is applied in the 'inner' loop on the test set. 

In [ ]:
# Classifier: kNN
# Preprocessing: Univariate Analysis
# Dataset: test
# Use of GridSearchCV to determine k

clf_knn = KNeighborsClassifier()

parameters = {"n_neighbors": list(range(1, 50, 2))}

cv_4fold = StratifiedKFold(n_splits=4, shuffle=True)
grid_search = GridSearchCV(clf_knn, parameters, cv=cv_4fold, scoring='roc_auc')

train_all = []
test_all = []
for _ in range(4):
    for train_index, test_index in cv_4fold.split(data, y):
        train_scores = []
        test_scores = []
        X_test = data.iloc[test_index]
        y_test = y[test_index]
        X_train = data.iloc[train_index]
        y_train = y[train_index]

        # Data preprocessing
        X_train_uni, X_test_uni = data_preprocessing_uni(X_train, y_train, X_test)

        grid_search.fit(X_train_uni, y_train) 
        clf_knn = grid_search.best_estimator_

        # Test
        train_proba = clf_knn.predict_proba(X_train_uni)[:, 1]
        test_proba = clf_knn.predict_proba(X_test_uni)[:, 1]

        score_train = roc_auc_score(y_train, train_proba)
        score_test = roc_auc_score(y_test, test_proba)

        train_scores.append(score_train)
        test_scores.append(score_test)

    train_all.append(train_scores)
    test_all.append(test_scores)

# Create numpy array of scores and calculate the mean and std
all_train = np.array(train_all)
all_test = np.array(test_all)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

test_scores_mean = all_test.mean(axis=0)
test_scores_std = all_test.std(axis=0)
print(f'AUC: {test_scores_mean} +- {test_scores_std}')

The results using Univariate Analysis as a preprocessing step are better than using PCA.

# Logistic Regression

In [ ]:
# Classifier: Logistic Regression
# Preprocessing: PCA
# Dataset: train & validation 
# Use of GridsearchCV for parameter tuning

# Set up the GridSearch with a set of parameters
cv_4fold = StratifiedKFold(n_splits=4, shuffle=True)

grid_param = {'penalty' : ['l1', 'l2'], 'C' : np.logspace(-4, 4, 20),'solver' : ['liblinear']}
grid_search = GridSearchCV(LogisticRegression(), param_grid=grid_param, cv=cv_4fold,n_jobs=-1,scoring='roc_auc') 

X_train_pca, _ = data_preprocessing_pca(split_X_train, split_X_test)
grid_search.fit(X_train_pca, split_y_train)

# Show the complete results of the cross validation
pd.DataFrame(grid_search.cv_results_)
print(f'Best GridSearchCV of Logistic Regression within the 4-fold cross-validation for PCA preprocessing: {grid_search.best_score_}')

title = 'Learning curve for Logistic Regression classifier'
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
plot_learning_curve(grid_search.best_estimator_, title, X_train_pca, split_y_train, ax, ylim=(0.3, 1.01), cv=cv_4fold)

When using PCA in the preprocessing method, a lot of variance can still be seen from the learning curve, both in the graph as when we run it multiple times. 

To generalize better, we tried a Univariate feature selection below.

In [ ]:
# Classifier: Logistic Regression
# Preprocessing: Univariate Analysis
# Dataset: train & validation 
# Use of GridsearchCV for parameter tuning

# Set up the GridSearch with a set of parameters
cv_4fold = StratifiedKFold(n_splits=4, shuffle=True)

grid_param = {'penalty' : ['l1', 'l2'],'C' : np.logspace(-4, 4, 20),'solver' : ['liblinear']}
grid_search = GridSearchCV(LogisticRegression(), param_grid=grid_param, cv=cv_4fold,n_jobs=-1,scoring='roc_auc') 

# Data preprocessing 
X_train_uni, _ = data_preprocessing_uni(split_X_train, split_y_train, split_X_test)
grid_search.fit(X_train_uni, split_y_train)

# Show the complete results of the cross validation
pd.DataFrame(grid_search.cv_results_)
print(f'Best GridSearchCV of Logistic Regression within the 4-fold cross-validation for Univariate preprocessing: {grid_search.best_score_}')

title = 'Learning curve for Logistic Regression classifier'
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
plot_learning_curve(grid_search.best_estimator_, title, X_train_uni, split_y_train, ax, ylim=(0.3, 1.01), cv=cv_4fold)

The use of Univariate Analysis seems to improve the AUC score. This can both be seen in the attribute of GridSearchCV.best_score_ as can be seen in the learning curve graph where the training and validation error seem to converge within the sample size that we have. Next, we apply the method on the test data.

In [ ]:
# Classifier: Logistic Regression
# Preprocessing: Univariate Analysis
# Dataset: test
# Use of GridsearchCV for parameter tuning

cv_4fold = model_selection.StratifiedKFold(n_splits=4, shuffle=True)
grid_param = {'penalty' : ['l1', 'l2'],'C' : np.logspace(-4, 4, 20),'solver' : ['liblinear']}
grid_search = GridSearchCV(LogisticRegression(),param_grid=grid_param, cv=cv_4fold,n_jobs=-1, scoring='roc_auc') 

train_all = []
test_all = []
for _ in range(4):
    for train_index, test_index in cv_4fold.split(data, y):
        train_scores = []
        test_scores = []
        X_test = data.iloc[test_index]
        y_test = y[test_index]
        X_train = data.iloc[train_index]
        y_train = y[train_index]

        # Data preprocessing
        X_train_uni, X_validation_uni = data_preprocessing_uni(X_train, y_train, X_test)

        grid_search.fit(X_train_uni, y_train) 
        clf_lr = grid_search.best_estimator_
        
        # Test
        train_proba = clf_lr.predict_proba(X_train_uni)[:, 1]
        test_proba = clf_lr.predict_proba(X_validation_uni)[:, 1]

        score_train = metrics.roc_auc_score(y_train, train_proba)
        score_test = metrics.roc_auc_score(y_test, test_proba)

        train_scores.append(score_train)
        test_scores.append(score_test)

    train_all.append(train_scores)
    test_all.append(test_scores)

# Create numpy array of scores and calculate the mean and std
all_train = np.array(train_all)
all_test = np.array(test_all)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

test_scores_mean = all_test.mean(axis=0)
test_scores_std = all_test.std(axis=0)
print(f'The resulting method gives an AUC of {test_scores_mean} +- {test_scores_std}')

# Appendix
1. Preprocessing
2. Classifiers

In [ ]:
def data_preprocessing(X_train, y_train):
    '''Data preprocessing: Scaling, RFECV, PCA, Imputation missing data'''

    # 1. Scaling
    scaler = preprocessing.StandardScaler()
    # scaler = preprocessing.MinMaxScaler()
    # scaler = preprocessing.RobustScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)

    # 2. Feature selection/extraction
    # Create the Recursive Feature Elimination object and compute a cross-validated score.
    svc = svm.SVC(kernel="linear")

    rfecv = feature_selection.RFECV(estimator=svc, step=1, cv=model_selection.StratifiedKFold(4), scoring='roc_auc')
    rfecv.fit(X_train_scaled, y_train)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    print("Optimal number of features : %d" % rfecv.n_features_)

    # 3. PCA
    n_selected_features = rfecv.n_features_
    n_samples = X_train.shape
    n_components == min(n_samples, n_selected_features)
    pca = decomposition.PCA(n_components)
    pca.fit(X_train_scaled)
    X_train_pca = pca.transform(X_train_scaled)

    # 4. Imputation missing data
    missing_values=[0.0, 1.0]
    for elem in missing_values:
        imputer = KNNImputer(missing_values=elem, n_neighbors=5, weights='uniform')
        X_train_imputed = imputer.fit_transform(X_train_pca)

    return X_train_imputed

# 1.b PCA
To determine the number of components for PCA we used the Cumulative Summation of the Explained Variance. To retain 95% of the variance, 30 components are needed.

In [ ]:
# Determining number of components for PCA

# Loading of the data 
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of features: {len(data.columns)-1}')
y_labels = data['label']
del data['label']

y = sklearn.preprocessing.label_binarize(y_labels, ['T12', 'T34']) 
# 0 now stands for T12 and 1 for T34
y = [i[0] for i in y]
y = np.array(y)

cv_4fold = model_selection.StratifiedKFold(n_splits=4, shuffle=True)
split_X_train, split_X_test, split_y_train, split_y_test = train_test_split(data, y, stratify=y,test_size=0.2)

# Loop over the folds
for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
    X_validation = split_X_train.iloc[validation_index]
    y_validation = split_y_train[validation_index]
    X_train = split_X_train.iloc[training_index]
    y_train = split_y_train[training_index]

    # Scale data
    scaler = preprocessing.StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_validation_scaled = scaler.transform(X_validation)

    #PCA
    pca = PCA().fit(X_train_scaled)

# Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance')
plt.show()

# 2.a Random Forest Clasifier
We also tried the Random Forest Classifier. 
Summary of this classifier: The code below shows the set-up for the random forest classifier. First, the random forest classifier with a PCA preprocessing is applied. Despite tuning of the hyper parameters of the random forest, a lot of variance was present in the accuracy. Then, we performed the random forest classifier with the Univariate Analysis as preprocessing step. This resulted in a slightly higher mean accuracy, but still with a large variance. We concluded that the random forest classifier is not the optimal choice for our dataset. The model is to complex for our dataset.

In [ ]:
# Data preprocessing function for Random Forest classifier


def data_preprocessing_uni_rf(X_train, y_train, X_validation):
    '''Data preprocessing: first scaling and then univariate analysis'''

    # 1. Scaling 
    scaler = preprocessing.RobustScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    df_X_train_scaled = pd.DataFrame(X_train_scaled)
    X_validation_scaled = scaler.transform(X_validation)
    df_X_validation_scaled = pd.DataFrame(X_validation_scaled)

    bestfeatures = SelectKBest(mutual_info_classif, k=20)
    fit = bestfeatures.fit(df_X_train_scaled, y_train)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(df_X_train_scaled.columns)
    # Concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  # Naming the dataframe columns
    best_features = featureScores.nlargest(20,'Score')['Specs']

    X_train_uni = df_X_train_scaled[best_features]
    X_validation_uni = df_X_validation_scaled[best_features]

    return X_train_uni, X_validation_uni

In [ ]:
# Classifier: Random Forest
# Preprocessing: PCA
# Dataset: train & validation
# Use of GridsearchCV for parameter tuning

# Loading of the data 
data = load_data()
y_labels = data['label']
del data['label']

y = preprocessing.label_binarize(y_labels, ['T12', 'T34']) 
# 0 stands for T12 and 1 for T34
y = [i[0] for i in y]
y = np.array(y)

# Split data in a train and test set
split_X_train, split_X_test, split_y_train, split_y_test = train_test_split(data, y, stratify=y, test_size=0.2)

cv_4fold = model_selection.StratifiedKFold(n_splits=4, shuffle=True)

# Loop over the folds
num = 0
fig = plt.figure(figsize=(15, 15))
for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
    X_validation = split_X_train.iloc[validation_index]
    y_validation = split_y_train[validation_index]
    X_train = split_X_train.iloc[training_index]
    y_train = split_y_train[training_index]

    # Preprocessing
    X_train_pca, X_validation_pca = data_preprocessing_pca(X_train, X_validation)

    # Random Forest Classification
    k = 4
    skf = StratifiedKFold(k, random_state=None)

    # Tuning the hyperparameters
    grid_param = {'n_estimators': [10, 50, 100, 200, 400],'criterion': ['gini', 'entropy'],'bootstrap': [True, False]}
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid=grid_param, scoring='roc_auc', cv=skf, n_jobs=-1, verbose=2) 
    grid_search.fit(X_train_pca, y_train)

    best_hyperparameters = grid_search.best_params_

    # Best hyperparameters
    n_estimators = best_hyperparameters.get('n_estimators')
    criterion = best_hyperparameters.get('criterion')
    bootstrap = best_hyperparameters.get('bootstrap')
    print(f'Best number of trees: {(n_estimators)}')
    print(f'Best split quality function: {(criterion)}')
    print(f'Best bootstrap: {(bootstrap)}')
    best_result = grid_search.best_score_

    # Apply classifier with tuned hyperparameters
    classifier = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, bootstrap=bootstrap)
    classifier.fit(X_train_pca, y_train)

    # Calculate accuracy
    classifier_predictions_test = classifier.predict(X_validation_pca)
    accuracy = metrics.accuracy_score(y_validation, classifier_predictions_test)
    print(f'Accuracy: {(accuracy)}')
    print('-'*80)

    # Learning curve
    title = 'Learning curve for Random Forest Classifier'
    ax = fig.add_subplot(2, 2, num + 1)
    plot_learning_curve(classifier, title, X_train_pca, y_train, ax, ylim=(0.3, 1.01), cv=skf)
    num += 1

One can see in the results above, the accuracy is varying a lot between the folds. It can differ from approximately 0.55 to 0.80. It can be concluded that the random forest method with PCA preprocessing is not robust for this dataset.

In the section below, the random forest classifier with Univariate Analysis as preprocessing is conducted.

In [ ]:
# Classifier: Random Forest
# Preprocessing: Univariate Analysis
# Dataset: train & validation
# Use of GridsearchCV for parameter tuning

# Loading of the data 
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of features: {len(data.columns)-1}')
y_labels = data['label']
del data['label']

y = preprocessing.label_binarize(y_labels, ['T12', 'T34']) 
# 0 stands for T12 and 1 for T34
y = [i[0] for i in y]
y = np.array(y)

# Split data in a train and test set
split_X_train, split_X_test, split_y_train, split_y_test = train_test_split(data, y, stratify=y, test_size=0.2)

cv_4fold = model_selection.StratifiedKFold(n_splits=4, shuffle=True)

# Loop over the folds
num = 0
fig = plt.figure(figsize=(15, 15))
for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
    train_scores = []
    test_scores = []
    X_train = split_X_train.iloc[training_index]
    y_train = split_y_train[training_index]
    X_validation = split_X_train.iloc[validation_index]
    y_validation = split_y_train[validation_index]

    # Preprocessing
    X_train_uni, X_validation_uni = data_preprocessing_uni_rf(X_train, y_train, X_validation)

    # Random Forest Classification
    k = 4
    skf = StratifiedKFold(k, random_state=None) 

    # Tuning the hyperparameters
    grid_param = {'n_estimators': [10, 50, 100, 200, 400],'criterion': ['gini', 'entropy'],'bootstrap': [True, False]}
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid=grid_param, scoring='roc_auc', cv=skf, n_jobs=-1, verbose=2)   
    grid_search.fit(X_train_uni, y_train)

    best_hyperparameters = grid_search.best_params_

    # Best hyperparameters
    n_estimators = best_hyperparameters.get('n_estimators')
    criterion = best_hyperparameters.get('criterion')
    bootstrap = best_hyperparameters.get('bootstrap')
    print(f'Best number of trees: {(n_estimators)}')
    print(f'Best split quality function: {(criterion)}')
    print(f'Best bootstrap: {(bootstrap)}')
    best_result = grid_search.best_score_  

    # Apply classifier with tuned hyperparameters
    classifier = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, bootstrap=bootstrap)
    classifier.fit(X_train_uni, y_train)

    # Calculate accuracy
    classifier_predictions_test = classifier.predict(X_validation_uni)
    accuracy = metrics.accuracy_score(y_validation, classifier_predictions_test)
    print(f'Accuracy: {(accuracy)}')
    print('-'*80)

    # Learning curve
    title = 'Learning curve for Random Forest Classifier'
    ax = fig.add_subplot(2, 2, num + 1)
    plot_learning_curve(classifier, title, X_train_uni, y_train, ax, ylim=(0.3, 1.01), cv=skf)
    num += 1

As one can see in the results above, the accuracy is varying a lot. Random Forest with PCA as preprocessing results in a lower mean accuracy than Random Forest with Univariate Analysis as preprocessing. 

The overall conclusion is that the Random Forest classifier is not suitable for out dataset.

# 2.b Support Vector Machine (SVM)


In [ ]:
# Classifier: SVM
# Preprocessing: PCA
# Dataset: train & validation
# Use of GridsearchCV for parameter tuning

# Loading of the data 
data = load_data()
y_labels = data['label']
del data['label']

y = preprocessing.label_binarize(y_labels, ['T12', 'T34']) 
# 0 stands for T12 and 1 for T34
y = [i[0] for i in y]
y = np.array(y)

# Split data in a train and test set
split_X_train, split_X_test, split_y_train, split_y_test = train_test_split(data, y, stratify=y, test_size=0.2)

cv_4fold = model_selection.StratifiedKFold(n_splits=4, shuffle=True)

# Loop over the folds
for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
    X_validation = split_X_train.iloc[validation_index]
    y_validation = split_y_train[validation_index]
    X_train = split_X_train.iloc[training_index]
    y_train = split_y_train[training_index]

    # Preprocessing
    X_train_pca, X_validation_pca = data_preprocessing_pca(X_train, X_validation)

    # Hyperparameter tuning
    grid_param =[{'kernel': ('linear','rbf'), 'gamma': [1e-04, 1e-03, 1e-02, 0.1, 1, 10],'C': [1e-04, 1e-03, 1e-02, 0.1, 1, 10]}]
    grid_search=GridSearchCV(SVC(),grid_param,n_jobs=-1,verbose=2)
    grid_search.fit(X_train_pca, y_train)

    best_hyperparameters = grid_search.best_params_

    kernel=best_hyperparameters.get('kernel')
    gamma=best_hyperparameters.get('gamma')
    C=best_hyperparameters.get('C')

    print(kernel)
    print(gamma)
    print(C)
    best_result = grid_search.best_score_  

    # Classification with tuned hyperparameters
    clf_svm = svm.SVC(kernel=kernel, gamma=gamma, C=C)
    clf_svm.fit(X_train_pca, y_train)

    # Calculate accuracy
    clf_svm_predictions = clf_svm.predict(X_validation_pca)

    accuracy = metrics.accuracy_score(y_validation, clf_svm_predictions)
    print(f'Accuracy: {(accuracy)}')
    print('-'*80)

    # Learning curve
    title = 'Learning curve for SVM classifier'
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    plot_learning_curve(clf_svm, title, X_train_pca, y_train, ax, ylim=(0.3, 1.01), cv=cv_4fold)

In [ ]:
# Classifier: SVM
# Preprocessing: Univariate Analysis
# Dataset: train & validation
# Use of GridsearchCV for parameter tuning

# Loading of the data 
data = load_data()
y_labels = data['label']
del data['label']

y = preprocessing.label_binarize(y_labels, ['T12', 'T34']) 
# 0 stands for T12 and 1 for T34
y = [i[0] for i in y]
y = np.array(y)

# Split data in a train and test set
split_X_train, split_X_test, split_y_train, split_y_test = train_test_split(data, y, stratify=y, test_size=0.2)

cv_4fold = model_selection.StratifiedKFold(n_splits=4, shuffle=True)

# Loop over the folds
for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
    X_validation = split_X_train.iloc[validation_index]
    y_validation = split_y_train[validation_index]
    X_train = split_X_train.iloc[training_index]
    y_train = split_y_train[training_index]

    # Preprocessing
    X_train_uni, X_validation_uni = data_preprocessing_uni(X_train, y_train, X_validation)

    # Hyperparameter tuning
    grid_param =[{'kernel': ('linear','rbf'), 'gamma': [1e-04, 1e-03, 1e-02, 0.1, 1, 10],'C': [1e-04, 1e-03, 1e-02, 0.1, 1, 10]}]
    grid_search=GridSearchCV(SVC(),grid_param,n_jobs=-1,verbose=2)
    grid_search.fit(X_train_uni, y_train)

    best_hyperparameters = grid_search.best_params_

    kernel=best_hyperparameters.get('kernel')
    gamma=best_hyperparameters.get('gamma')
    C=best_hyperparameters.get('C')

    print(kernel)
    print(gamma)
    print(C)
    best_result = grid_search.best_score_  

    # Classification with tuned hyperparameters
    clf_svm = svm.SVC(kernel=kernel, gamma=gamma, C=C)
    clf_svm.fit(X_train_uni, y_train)

    # Calculate accuracy
    clf_svm_predictions = clf_svm.predict(X_validation_uni)

    accuracy = metrics.accuracy_score(y_validation, clf_svm_predictions)
    print(f'Accuracy: {(accuracy)}')
    print('-'*80)

    # Learning curve
    title = 'Learning curve for SVM classifier'
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    plot_learning_curve(clf_svm, title, X_train_uni, y_train, ax, ylim=(0.3, 1.01), cv=cv_4fold)